In [45]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import models
import metrics

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
def get_images(amount, factor):
    images = []
    file_path = f"../Datasets/Test/Set{amount}/image_SRF_{factor}/"
    for i in range(1, amount + 1):
        hr_image = Image.open(f"{file_path}/img_{i:03d}_SRF_{factor}_HR.png")
        lr_image = Image.open(f"{file_path}/img_{i:03d}_SRF_{factor}_LR.png")
        images.append((hr_image, lr_image))
    return images

In [47]:
amount = 5
factor = 2
images = get_images(amount, factor)
print(images[0][0].size, images[0][1].size)

(512, 512) (256, 256)


In [48]:
lr_images = [img[1] for img in images]
hr_images = [img[0] for img in images]

In [49]:
methods = ["bicubic", "nearest", "bilinear"]
for method in methods:
    print(f"Method: {method}")
    upsampled_imgs = models.upsample_images(lr_images, factor, method)
    psnr = np.mean([metrics.calculate_psnr(hr_images[i], upsampled_imgs[i]) for i in range(amount)])
    ssim = np.mean([metrics.calculate_ssim(hr_images[i], upsampled_imgs[i]) for i in range(amount)])
    print(f"PSNR: {psnr:.2f}, SSIM: {ssim:.4f}")

Method: bicubic
PSNR: 31.79, SSIM: 0.9178
Method: nearest
PSNR: 29.08, SSIM: 0.8913
Method: bilinear
PSNR: 30.42, SSIM: 0.8974
